In [1]:
import requests
import geopandas as gpd
import pandas as pd
import time
import os
import json

In [ ]:
# gets all the neighbourhoods, but what are the neighbourhoods?
url = "https://data.police.uk/api/leicestershire/neighbourhoods"
url = "https://data.police.uk/api/leicestershire/neighbourhoods"
url = "https://data.police.uk/api/leicestershire/neighbourhoods"
url = "https://data.police.uk/api/crime-categories"
# url = "https://data.police.uk/api/forces/leicestershire"

# # gets all the forces
# url = "https://data.police.uk/api/forces"
resp = requests.get(url)
resp.raise_for_status()
response = resp.json()

In [ ]:
pd.DataFrame(response)

In [2]:

def load_geojson(path: str):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

boundaries = load_geojson(os.path.join('data', "london_neighborhoods.geojson"))

In [ ]:
# polygon = ":".join(f"{lat:.6f},{lng:.6f}" for lng, lat in boundaries["features"][0]["geometry"]["coordinates"][0][:-1])

# polygon_dict = {}



[[-0.03703838666761, 51.524248378983],
 [-0.037253917706314, 51.524161985588],
 [-0.038035302297156, 51.523794690118],
 [-0.038354124822786, 51.523619312946],
 [-0.038612166406022, 51.523486893086],
 [-0.0388050624447, 51.523393023512],
 [-0.039063390448756, 51.523284060307],
 [-0.039338716386037, 51.523186090794],
 [-0.039610491912455, 51.523098924455],
 [-0.039873747341452, 51.523013422718],
 [-0.040432081842009, 51.52283476569],
 [-0.040886347173581, 51.522696714991],
 [-0.041471329408425, 51.522567040289],
 [-0.042062985740007, 51.522451061967],
 [-0.042571792734203, 51.522351576088],
 [-0.042776522202233, 51.522319928786],
 [-0.043041565189459, 51.522289286996],
 [-0.043405722856346, 51.522267491369],
 [-0.043836272885283, 51.522246801413],
 [-0.044064690502493, 51.522230834214],
 [-0.044201933048926, 51.52222322986],
 [-0.044468722950098, 51.522188117702],
 [-0.044907209227953, 51.522115417006],
 [-0.045667995045629, 51.521987751618],
 [-0.046201951879293, 51.521872581477],
 [-0.

In [11]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
dateutil_available = True


def get_street_crimes(category: str, date: str, polygon: str):
    url = f"https://data.police.uk/api/crimes-street/{category}?date={date}&poly={polygon}"
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json()


def generate_month_pairs(start: str, end: str):
    """
    Generate a list of year-month strings between 'start' and 'end' inclusive.
    Both `start` and `end` should be in 'YYYY-MM' format.
    """
    pairs = []
    if dateutil_available:
        current = datetime.strptime(start, '%Y-%m')
        last = datetime.strptime(end, '%Y-%m')
        while current <= last:
            pairs.append(current.strftime('%Y-%m'))
            current += relativedelta(months=1)
    else:
        pairs = [end]
    return pairs


def save_json(path: str, data):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


def get_polygon_url(coordinates: str):
    return ":".join(f"{lat:.6f},{lng:.6f}" for lng, lat in coordinates)

# test = get_street_crimes("burglary", "2024-01", polygon)
# generate_month_pairs("2023-01", datetime.now().strftime("%Y-%m"))
neighborhoods_boundaries = [{"coordinates": features["geometry"]["coordinates"][0][:-1], **features["properties"]} for features in boundaries["features"]]

import itertools
from tqdm.notebook import tqdm

for month, neighborhood in tqdm(itertools.product(generate_month_pairs("2023-01", datetime.now().strftime("%Y-%m")), neighborhoods_boundaries)):
    print(month, neighborhood["name"])
    crimes = get_street_crimes("burglary", month, get_polygon_url(neighborhood["coordinates"]))
    save_json(os.path.join("data", "street_crimes", month, f"{neighborhood['name']}.json"), crimes)
    time.sleep(0.5)


0it [00:00, ?it/s]

2023-01 Bethnal Green East
2023-01 Blackwall and Cubitt Town


HTTPError: 414 Client Error: Request-URI Too Large for url: https://data.police.uk/api/crimes-street/burglary?date=2023-01&poly=51.490823,-0.000451:51.490875,-0.003025:51.490876,-0.003307:51.490876,-0.003378:51.490876,-0.003497:51.490877,-0.003572:51.490878,-0.003648:51.490878,-0.003728:51.490879,-0.003807:51.490879,-0.003883:51.490879,-0.003954:51.490879,-0.004036:51.490879,-0.004111:51.490880,-0.004190:51.490880,-0.004269:51.490880,-0.004328:51.490881,-0.004504:51.490881,-0.004566:51.490882,-0.004680:51.490794,-0.004698:51.490744,-0.004716:51.490805,-0.005068:51.490921,-0.005731:51.490950,-0.005851:51.491046,-0.005815:51.491337,-0.005675:51.491511,-0.005591:51.491585,-0.005601:51.491628,-0.005616:51.491685,-0.005657:51.491841,-0.005812:51.492215,-0.006181:51.492260,-0.006250:51.492428,-0.006413:51.492612,-0.006614:51.492910,-0.006892:51.492967,-0.006961:51.492866,-0.007220:51.492838,-0.007274:51.492629,-0.007369:51.492621,-0.007512:51.492666,-0.007719:51.492585,-0.007760:51.492458,-0

In [ ]:
import requests
import os
import json
from datetime import datetime
from requests.exceptions import HTTPError

# Optional: install python-dateutil for full month iteration
try:
    from dateutil.relativedelta import relativedelta
    dateutil_available = True
except ImportError:
    dateutil_available = False

BASE_URL = 'https://data.police.uk/api'
ALL_CRIME_CATEGORY = 'all-crime'

# Iterate months between two YYYY-MM strings
def months_between(start: str, end: str):
    if dateutil_available:
        start_dt = datetime.strptime(start, '%Y-%m')
        end_dt = datetime.strptime(end, '%Y-%m')
        dt = start_dt
        while dt <= end_dt:
            yield dt.strftime('%Y-%m')
            dt += relativedelta(months=1)
    else:
        yield end


def format_poly(coords: list) -> str:
    """Convert list of [lat, lng] pairs to API poly parameter format."""
    return ':'.join(f"{lat:.6f},{lng:.6f}" for lat, lng in coords)


def get_street_crimes(polygon: list, date: str):
    """Fetch all street-level crimes ('all-crime') within polygon for a month using POST."""
    url = f"{BASE_URL}/crimes-street/{ALL_CRIME_CATEGORY}"
    poly_str = format_poly(polygon)
    payload = {'date': date, 'poly': poly_str}
    try:
        r = requests.post(url, json=payload)
        r.raise_for_status()
    except HTTPError as e:
        status = getattr(e.response, 'status_code', None)
        if status in (503, 400):
            print(f"Warning: No data ({status}) for {date}, polygon too large/complex.")
            return []
        print(f"Error fetching crimes: {e}")
        return []
    return r.json()


def get_outcomes(persistent_id: str):
    """Fetch outcomes for a crime."""
    try:
        r = requests.get(f"{BASE_URL}/outcomes-for-crime/{persistent_id}")
        r.raise_for_status()
        return r.json().get('outcomes', [])
    except HTTPError:
        return []


def save_json(path: str, data):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)




def main(geojson_path: str, start_month: str = '2011-07'):
    # Determine latest month
    try:
        last_date = requests.get(f"{BASE_URL}/crime-last-updated").json().get('date')
        last_month = last_date[:7]
    except Exception:
        last_month = datetime.now().strftime('%Y-%m')

    # Load regions
    gj = load_geojson(geojson_path)
    features = gj.get('features', [])

    for feat in features:
        props = feat.get('properties', {})
        region = props.get('name') or props.get('code')
        coords = feat['geometry']['coordinates'][0]
        poly = [[lat, lng] for lng, lat in coords]

        for month in months_between(start_month, last_month):
            print(f"Fetching crimes for {region} in {month}")
            crimes = get_street_crimes(poly, month)
            save_json(f"data/{region}/{month}/crimes.json", crimes)

            outcomes = {}
            for crime in crimes:
                if pid := crime.get('persistent_id'):
                    outcomes[pid] = get_outcomes(pid)
            save_json(f"data/{region}/{month}/outcomes.json", outcomes)

if __name__ == '__main__':
    path = os.path.join('data', "london_neighborhoods.geojson")
    main(path)
